In [1]:
# import the relevant libraries
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import PolynomialFeatures
from tabulate import tabulate
from statsmodels.tools import add_constant
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from scipy import stats

In [2]:
# Load the data 
df = pd.read_csv('Olist_remove_no_correlated_no_item2.csv')
df.head()

,product_id,item_price,freight_value,payment_sequential,payment_type,payment_installments,order_purchase_timestamp,review_score,postal_code,state,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,sales_revenue
0,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,1,1,2,2017-09-13 08:59:00,5,28013,18,cool_stuff,650.0,28.0,9.0,14.0,58.9
1,4244733e06e7ecb4970a6e2683c13e61,55.9,17.96,1,0,1,2017-06-28 11:52:00,5,75800,8,cool_stuff,650.0,28.0,9.0,14.0,55.9
2,4244733e06e7ecb4970a6e2683c13e61,64.9,18.33,1,1,2,2018-05-18 10:25:00,4,30720,10,cool_stuff,650.0,28.0,9.0,14.0,64.9
3,4244733e06e7ecb4970a6e2683c13e61,58.9,16.17,1,1,3,2017-08-01 18:38:00,5,83070,17,cool_stuff,650.0,28.0,9.0,14.0,58.9
4,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,1,1,4,2017-08-10 21:48:00,5,36400,10,cool_stuff,650.0,28.0,9.0,14.0,58.9


In [3]:
from scipy import stats
import numpy as np

# Select the columns you're interested in
cols = [ 'item_price']

# Calculate z-scores
z_scores = np.abs(stats.zscore(df[cols]))

# Create a new DataFrame with the z-scores
df_z_scores = pd.DataFrame(z_scores, columns=cols, index=df.index)

# Join the z-scores DataFrame with the original DataFrame
df = df.join(df_z_scores, rsuffix='_zscore')

# Keep only rows where all z-scores are less than 3
df = df[(df[[col + '_zscore' for col in cols]] < 3).all(axis=1)]

# Now you can drop the z-score columns
df = df.drop(columns=[col + '_zscore' for col in cols])

In [4]:
# Drop the defined columns from the dataframe to form feature set 'X'.
X = df.drop(['sales_revenue', 'order_purchase_timestamp'], axis=1)

# Assign 'sales_revenue' column as the target variable 'y'.
y = df['sales_revenue']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# Keep a copy of the Product IDs for the test set before dropping from X
X_Product_IDs = X['product_id'].copy()

# Drop the Product ID from X
X = X.drop('product_id', axis=1)

# Split the data into train/test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Split train_val set into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=0)

# Define preprocessing for numerical columns (scale them)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

# Create preprocessing and training pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the pipeline to train a linear regression model on the training set
pipeline.fit(X_train, y_train)

# Make predictions on the test set using the fitted model
y_pred_test = pipeline.predict(X_test)

# Create a DataFrame for the predictions, indexed by Product ID
df_predictions = pd.DataFrame({
    'product_id': X_Product_IDs.loc[X_test.index],
    'Actual': y_test,
    'Predicted': y_pred_test
})

print(df_predictions)

                              product_id  Actual   Predicted
107507  aeb767ca82c5a6cca8bbac33c4e21579  136.00   50.212283
40414   0da9ffd92214425d880de3f94e74ce39  112.00  133.055808
51022   2f13d1dc8b4e1d9d8027be50339546a9  105.00  121.311643
82904   3b66a296f7f9d4a5aa79ca89f460af73   33.80   38.910909
105162  9523f1a3e7db9e38d55379435bd450c9  134.17  147.530085
...                                  ...     ...         ...
5577    389d119b48cf3043d311335e499d9c6b   99.80   55.492345
41975   6ec26071510e1cf38d6988f473f8d7a5   69.90   74.849187
48747   67bd616e1ba0d3d3e8545f3113b0140d   14.49   14.865700
35601   5ad2b7b30050dd8aa51c2fed1db13e22   99.00  114.671188
69018   730f1927f42fda4370209c28203eb0ab   55.96   12.980409

[22658 rows x 3 columns]


In [ ]:
from sklearn.metrics import r2_score

# Make predictions on the train, validation, and test sets
y_pred_train = pipeline.predict(X_train)
y_pred_val = pipeline.predict(X_val)
y_pred_test = pipeline.predict(X_test)

# Calculate R-squared scores
r2_train = r2_score(y_train, y_pred_train)
r2_val = r2_score(y_val, y_pred_val)
r2_test = r2_score(y_test, y_pred_test)

print('Train R2 score: ', r2_train)
print('Validation R2 score: ', r2_val)
print('Test R2 score: ', r2_test)

In [6]:
X.columns

Index(['item_price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'review_score', 'postal_code', 'state',
       'product_category_name', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm'],
      dtype='object')

In [ ]:
# Create a dataframe for the coefficients
coef_df = pd.DataFrame(pipeline.named_steps['regressor'].coef_, X.columns, columns=['Coefficient'])

# Append the intercept
coef_df = coef_df.append(pd.DataFrame([pipeline.named_steps['regressor'].intercept_], ['Intercept'], columns=['Coefficient']))

# Display the dataframe
print(coef_df)

In [8]:
df_compare = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
print(df_compare)

        Actual   Predicted
107507  136.00   50.212283
40414   112.00  133.055808
51022   105.00  121.311643
82904    33.80   38.910909
105162  134.17  147.530085
...        ...         ...
5577     99.80   55.492345
41975    69.90   74.849187
48747    14.49   14.865700
35601    99.00  114.671188
69018    55.96   12.980409

[22658 rows x 2 columns]


In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor


# vif = df
# vif["Feature"] = X.columns
# vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [10]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))

Mean Absolute Error: 25.355739731063732
Mean Squared Error: 4941.4963213432075
Root Mean Squared Error: 70.29577740763102


In [11]:
# Create a dataframe for test set predictions
df_test_predictions = pd.DataFrame(y_pred_test, columns=['Predicted'], index=y_test.index)

# Merge this dataframe with the actuals, using the common index
df_test_combined = y_test.to_frame().join(df_test_predictions)

# Now 'sales_revenue' is your actual values and 'Predicted' is the model's predictions
print(df_test_combined[['sales_revenue', 'Predicted']])

        sales_revenue   Predicted
107507         136.00   50.212283
40414          112.00  133.055808
51022          105.00  121.311643
82904           33.80   38.910909
105162         134.17  147.530085
...               ...         ...
5577            99.80   55.492345
41975           69.90   74.849187
48747           14.49   14.865700
35601           99.00  114.671188
69018           55.96   12.980409

[22658 rows x 2 columns]


In [12]:
print(y_test.shape)
print(y_pred_test.shape)

(22658,)
(22658,)


In [14]:
import numpy as np

# Assuming y_test and y_pred_test are your actual and predicted values

# Calculate MAPE
mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

# Calculate MPE
mpe = np.mean((y_test - y_pred_test) / y_test) * 100

# Calculate PRMSE
prmse = (np.sqrt(np.mean((y_test - y_pred_test) ** 2)) / np.mean(y_test)) * 100

# Print the metrics
print("Mean Absolute Percentage Error (MAPE): {:.2f}%".format(mape))
print("Mean Percentage Error (MPE): {:.2f}%".format(mpe))
print("Percentage Root Mean Squared Error (PRMSE): {:.2f}%".format(prmse))

Mean Absolute Percentage Error (MAPE): 23.53%
Mean Percentage Error (MPE): -12.01%
Percentage Root Mean Squared Error (PRMSE): 60.32%


In [15]:
from sklearn.metrics import mean_squared_error

# Assuming y_train and y_test are the actual target values
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print("Train MSE:", train_mse)
print("Test MSE:", test_mse)

NameError: name 'regressor' is not defined

In [16]:
from sklearn.metrics import mean_squared_error

train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

NameError: name 'train_mse' is not defined

In [17]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)  # Training the algorithm

# To retrieve the intercept:
print("Intercept:", regressor.intercept_)

# For retrieving the slope:
print("Coefficients:", regressor.coef_)

y_pred_train = regressor.predict(X_train)
y_pred_test = regressor.predict(X_test)

# Calculate and print the accuracy (R-squared) for train and test sets
print("Train R-squared:", r2_score(y_train, y_pred_train))
print("Test R-squared:", r2_score(y_test, y_pred_test))

# Calculate the residuals
residuals = y_test - y_pred_test


# Plotting the residuals against the predicted values with color-coded data points
plt.scatter(y_pred_test, residuals, alpha=0.5, c=residuals, cmap='coolwarm')
plt.axhline(y=0, color='red', linestyle='--')  # Add a horizontal line at y=0
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Predicted Values (Homoscedasticity)')
plt.colorbar(label='Residuals')  # Add a colorbar indicating the range of residuals
plt.show()

ValueError: could not convert string to float: 'sports_leisure'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming residuals is your residual data

# Create a DataFrame with residuals
residuals_data = pd.DataFrame({'Residuals': residuals})

# Count the occurrences of each residual value
residuals_counts = residuals_data['Residuals'].value_counts()

# Display the residuals counts as a table
print(residuals_counts)

In [ ]:
# plt.scatter(df['payment_value'], df['sales_revenue'])
# plt.xlabel('payment_value')
# plt.ylabel('sales_revenue')
# plt.title('Scatter plot of payment_value against sales_revenue')
# plt.show()

In [ ]:
from statsmodels.stats.stattools import durbin_watson

dw = durbin_watson(residuals)
print('Durbin-Watson:', dw)

In [ ]:
from scipy.stats import probplot

probplot(residuals, plot=plt)
plt.show()

In [ ]:
import pandas as pd

# Create a DataFrame with the predicted values and residuals
residuals_df = pd.DataFrame({'Predicted Values': y_pred_test, 'Residuals': residuals})

# Display the DataFrame
print(residuals_df)

In [ ]:
# Merge with 'order_purchase_timestamp' information
df_test_combined = df_test_combined.join(df['order_purchase_timestamp'])

# Convert 'order_purchase_timestamp' to datetime if it isn't already
df_test_combined['order_purchase_timestamp'] = pd.to_datetime(df_test_combined['order_purchase_timestamp'])

# Set 'order_purchase_timestamp' as the index
df_test_combined.set_index('order_purchase_timestamp', inplace=True)

# Sort the DataFrame by the index
df_test_combined.sort_index(inplace=True)

# Now df_test_combined contains both the actual and predicted sales revenue, in chronological order
print(df_test_combined[['sales_revenue', 'Predicted']])

In [ ]:
# Resample the dataframe to monthly frequency, aggregating with mean
df_monthly = df_test_combined.resample('M').mean()

# Create a line plot of actual and predicted values
plt.figure(figsize=(12,6))
plt.plot(df_monthly.index, df_monthly['sales_revenue'], label='Actual')
plt.plot(df_monthly.index, df_monthly['Predicted'], label='Predicted')

# Add labels, title, and legend
plt.xlabel('Time')
plt.ylabel('Sales Revenue')
plt.title('Actual vs Predicted Sales Revenue Over Time (Monthly)')
plt.legend()

# Show the plot
plt.show()